In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os
os.environ['OMP_NUM_THREADS'] = '1'

from pympler import asizeof #Para liberar memoria
import gc

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, to_rgb

from sklearn import metrics
from sklearn.cluster import KMeans
import kneed
import random

from sklearn.preprocessing import StandardScaler

semilla = 8

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_clean_afi_sel_g_all_2.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_l = pickle.load( archivo )     

In [4]:
data_l.shape #(62130167, 16)

(62130167, 16)

In [5]:
inicio = time.time()
data_l = data_l.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[ True, True, True] )
data_l.reset_index(inplace=True)
data_l.rename(columns={'index': 'nuevo_indice'}, inplace=True)
data_l.drop(columns=['nuevo_indice'], inplace=True)
data_l['INDICE'] = data_l.index

#Casos de no análisis
data_no_grupo = data_l[ (data_l['GRUPO_SEL']==0) ].copy()
data_no_grupo['ATI_CJ_M1'] = np.nan
data_no_grupo['ATI_CJ_M2'] = np.nan
data_no_grupo['ATI_CJ_M3'] = np.nan

#Casos de análisis
data = data_l[ (data_l['GRUPO_SEL']==1) ].copy()

fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 53.33992838859558  segundos


In [ ]:
#Se calcula los límites máximos y mínimos según el diagrama de caja y bigote
##Caso 1: Para toda la historia laboral
inicio = time.time()   
grupo1 = data_l.copy()
Q1 = grupo1.groupby('CEDULA_COD')['SALARIO'].quantile(0.25)
Q3 = grupo1.groupby('CEDULA_COD')['SALARIO'].quantile(0.75)
IQR = Q3-Q1
LI = Q1 - 1.5 * IQR
LS = Q3 + 1.5 * IQR
data['LS1'] = data['CEDULA_COD'].map( LS )

##Caso 2: Para la historia laboral a partir del año 2000
grupo2 = data[ data['ANIO']>=2000 ].copy()
Q1 = grupo2.groupby('CEDULA_COD')['SALARIO'].quantile(0.25)
Q3 = grupo2.groupby('CEDULA_COD')['SALARIO'].quantile(0.75)
IQR = Q3-Q1
LI = Q1 - 1.5 * IQR
LS = Q3 + 1.5 * IQR
data['LS2'] = data['CEDULA_COD'].map( LS )

fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos')

del grupo1, grupo2, Q1, Q3, IQR, LI, LS
gc.collect()


In [ ]:
data

In [ ]:
#Funciones para hacer gráficos
def graf_obs( data_filt, ced, tipo ):
    valores = data_filt[ced][ tipo ]
    plt.figure( figsize=(10, 6))
    plt.scatter(range(len(valores)), valores, color='blue', marker='o')

    # Etiquetas y título
    plt.ylabel("USD")
    plt.title("Gráfico del Salario")

    plt.grid(True)
    plt.show()

def graf_obs_kmean( data_filt, ced):
    valores = np.array([[i + 1, valor] for i, valor in enumerate(data_filt[ced]['SALARIO'])])
    n = data_filt[ced]['num_codo']
    
    # Aplicar KMeans
    kmeans = KMeans(n_clusters=n, init='k-means++', n_init=10, random_state=42)
    y_pred = kmeans.fit_predict(valores)
    
    plt.figure(figsize=(10, 6))
    
    # Colormap para los clusters
    cmap = plt.get_cmap("jet")
    
    scatter = plt.scatter(valores[:, 0], valores[:, 1], c=y_pred, cmap=cmap, label='_nolegend_')
    
    
    plt.scatter(kmeans.cluster_centers_[:, 0],
                kmeans.cluster_centers_[:, 1],
                marker='^',
                c=list(range(0,n)),
                s=100,
                linewidth=2,
                edgecolor='black',
                cmap="jet",
                label=[f'Clúster {i+1}' for i in range(n)])
    plt.xlabel("Datos")
    plt.ylabel("USD")
    plt.title("Gráfico de Clústeres con Centroides")
    plt.grid(True)
    
     # Crear manejadores de leyenda para los puntos
    handles, labels = scatter.legend_elements(prop="colors", alpha=0.6)
    
    # Añadir la leyenda para los puntos y los centroides
    plt.legend( handles = handles, labels = [f'Clúster {i+1}' for i in range(n)], loc='best')

    plt.show()

In [ ]:
data

In [ ]:
#Algortimos K-means
cedula_1 = data.groupby('CEDULA_COD')['NUM_SEC_MES'].apply( lambda x: (x != 1).any() )
cedula_dist = cedula_1[ cedula_1 ].index
ul = data[ ~data['CEDULA_COD'].isin( cedula_dist )]
ml = data[  data['CEDULA_COD'].isin( cedula_dist )]

In [ ]:
#Determinación del número de cluster según el método del codo y Silueta
# Normalización de datos
def data_norm( df1 ):
    df = df1.copy()
    me = df.groupby('CEDULA_COD')['SALARIO'].mean()
    st = df.groupby('CEDULA_COD')['SALARIO'].std(0)
    df['ME'] = df['CEDULA_COD'].map( me )
    df['ST'] = df['CEDULA_COD'].map( st )
    df['SALARIO_NORM'] = ( df['SALARIO'] - df['ME'] ) / df['ST'] 
    return df

#Determinación del número de cluster por el método de la silueta
def max_clust_silueta( silhouettes ):
    silhouettes = np.array(silhouettes)
    diferencias = np.abs(1 - silhouettes)
    indice_cercano = np.argmin( diferencias )
    return indice_cercano + 2

#Determinar el número de cluster por el método del codo
def num_cluster( data, num, num_sample, tipo, duplicados ):
    print('*' * 102)
    print('Determinación del número de clúster por el método del codo y silueta')
    inicio = time.time()
    
    data_si_dic = data.copy()
    
    if( num_sample != 'todo' ):
        sample_size = num_sample
        sample_keys = random.sample( list(data_si_dic.keys()), min(sample_size, len(data_si_dic))) #Seleccion de cedulas
        data_filt = {key: data_si_dic[key] for key in sample_keys if key in data_si_dic}
    if (num_sample == 'todo' ):
        data_filt = data_si_dic
        
    for ced  in data_filt:
        
        if( duplicados=='si'):
            aux = np.array( data_filt[ced][ tipo ]).reshape(-1, 1)
        
        if( duplicados=='no'):
            aux = np.unique( np.array( data_filt[ced][ tipo ])).reshape(-1, 1)
        
        if len(aux) <= 1:
            data_filt[ced]['num_codo'] = 1
            data_filt[ced]['max_silueta'] = 1
           
        
        tf = min(num, len(aux) - 1) + 1
        
        distorsion = []
        silueta = []

        for i in range( 2, tf ):
            km = KMeans( i, init = 'k-means++', n_init = 1, max_iter = 300, tol = 1e-4, random_state = semilla )
            clustering = km.fit_predict( aux )

            if( len( np.unique( clustering ) ) > 1):
                distorsion.append( km.inertia_ )
                silueta.append( metrics.silhouette_score(aux, clustering))
            
            else:
                distorsion.append(0)
                silueta.append(0)
    
        if( len(distorsion) > 1 and len(silueta) > 1 ):
            data_filt[ced]['num_codo'] = kneed.KneeLocator(range(2, tf), distorsion[:tf], curve="convex", 
                                                             direction="decreasing").elbow or 1
    
            data_filt[ced]['max_silueta'] =  max_clust_silueta( silueta )
        
        else:
            data_filt[ced]['num_codo'] = 1
            data_filt[ced]['max_silueta'] = 1

    fin = time.time()  
    print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
    print('*' * 102)
    return data_filt

In [ ]:
#Normalización de datos-Z-core
print('*' * 40, 'Normalización Z-Score ', '*' * 40)
inicio = time.time()
ul1 =  data_norm( ul )
ml1 =  data_norm( ml )
fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
print('*' * 102)

del ul, ml
gc.collect()

In [ ]:
print('Total de cédulas', data['CEDULA_COD'].nunique()) #442570
print('Cedulas que tienen un único sector', ul1['CEDULA_COD'].nunique()) #373069
print('Cedulas que tienen más de un único sector', ml1['CEDULA_COD'].nunique()) #69501

In [ ]:
ul_dic = ul1.groupby('CEDULA_COD').agg({'SALARIO': list,'SALARIO_NORM': list, 'INDICE': list}).to_dict(orient='index')
ml_dic = ml1.groupby('CEDULA_COD').agg({'SALARIO': list,'SALARIO_NORM': list, 'INDICE': list}).to_dict(orient='index')

del ul1, ml1
gc.collect()

In [ ]:
#Se aplica un muestreo para ver como funciona el algoritmo
num = 5 # Número de cluster por defecto--máximo 5 se hace el supuesto por ser los mejores años
muestra = 'todo' # muestra = 'todo' #Para que sea en todas las cedulas
t1='SALARIO'
#t2='SALARIO_NORM'
duplicados = 'si'
ul_dic_cluster1 = num_cluster( ul_dic, num, muestra, t1, duplicados ) 
ml_dic_cluster1 = num_cluster( ml_dic, num, muestra, t1, duplicados ) 

In [ ]:
nombre_archivo = 'viu_k_mean_cluster1_ul.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto1 = ul_dic_cluster1

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto1, archivo) 
    
nombre_archivo = 'viu_k_mean_cluster1_ml.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto2 = ml_dic_cluster1

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto2, archivo)  

In [ ]:
num = 5 
muestra = 'todo' 
t1='SALARIO'
#t2='SALARIO_NORM'
duplicados = 'no'
ul_dic_cluster2 = num_cluster( ul_dic, num, muestra, t1, duplicados )
ml_dic_cluster2 = num_cluster( ml_dic, num, muestra, t1, duplicados )

In [ ]:
nombre_archivo = 'viu_k_mean_cluster2_ul.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto3 = ul_dic_cluster2

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto3, archivo) 
    
nombre_archivo = 'viu_k_mean_cluster2_ml.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto4 = ml_dic_cluster2

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto4, archivo)  

In [ ]:
# Se redondea la variable 'SALARIO_NORM' a 4 decimales
for key, value in ul_dic.items():
    value['SALARIO_NORM'] = [ round(num, 4) for num in value['SALARIO_NORM'] ]
    
for key, value in ml_dic.items():
    value['SALARIO_NORM'] = [ round(num, 4) for num in value['SALARIO_NORM'] ]

In [ ]:
def valores_nan(data):
    result = {}
    for cedula, info in data.items():
        salario_norm_array = np.array(info['SALARIO_NORM'])
        if np.isnan(salario_norm_array).any() or np.isinf(salario_norm_array).any():
            result[cedula] = True
        else:
            result[cedula] = False
    return result


In [ ]:
# Obtener resultados
veri = valores_nan( ml_dic )
ced_nan = []
for cedula, has_na in veri .items():
    if has_na:
        ced_nan.append(cedula)
ced_nan

In [ ]:
ml_dic[2802556]

In [ ]:
#Se filtran los valores nan e infinitos para que no tenga problemas el kmeans
ul_dic_filt_no_nan = { key: value for key, value in ul_dic.items()
                      if not (np.isnan(value['SALARIO_NORM']).any() or np.isinf(value['SALARIO_NORM']).any())}

ml_dic_filt_no_nan = { key: value for key, value in ml_dic.items()
                      if not (np.isnan(value['SALARIO_NORM']).any() or np.isinf(value['SALARIO_NORM']).any())}

ul_dic_filt_si_nan = { key: value for key, value in ul_dic.items()
                       if any(np.isnan(x) or np.isinf(x) for x in value['SALARIO_NORM'])}
ml_dic_filt_si_nan = { key: value for key, value in ml_dic.items()
                       if any(np.isnan(x) or np.isinf(x) for x in value['SALARIO_NORM'])}

In [ ]:
num = 5 
muestra = 'todo'
#t1='SALARIO'
t2='SALARIO_NORM'
duplicados = 'si'
ul_dic_cluster3 = num_cluster( ul_dic_filt_no_nan , num, muestra, t2, duplicados )
ml_dic_cluster3 = num_cluster( ml_dic_filt_no_nan, num, muestra, t2, duplicados )

In [ ]:
nombre_archivo = 'viu_k_mean_cluster3_ul.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto5 = ul_dic_cluster3

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto5, archivo) 
    
nombre_archivo = 'viu_k_mean_cluster3_ml.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto6 = ml_dic_cluster3

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto6, archivo)  

In [ ]:
num = 5 
muestra = 'todo'
#t1='SALARIO'
t2='SALARIO_NORM'
duplicados = 'no'
ul_dic_cluster4 = num_cluster( ul_dic_filt_no_nan, num, muestra, t2, duplicados )
ml_dic_cluster4 = num_cluster( ml_dic_filt_no_nan, num, muestra, t2, duplicados )

In [ ]:
nombre_archivo = 'viu_k_mean_cluster4_ul.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto7 = ul_dic_cluster4

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto7, archivo) 
    
nombre_archivo = 'viu_k_mean_cluster4_ml.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto8 = ml_dic_cluster4

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto8, archivo)  

In [ ]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_k_mean_cluster1_ul.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    ul_dic_cluster1 = pickle.load( archivo ) 
    
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_k_mean_cluster1_ml.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    ml_dic_cluster1 = pickle.load( archivo ) 

In [ ]:
#Se crea la función para agregar el número de cluster a la data por el método del codo y silueta
def agregar_num_cluster( data, colname, data_dic1, data_dic2, tipo):
    aux1 = None
    aux2 = None
    aux = None
    all_aux = None
    
    aux1 = {key: {tipo: value[tipo]} for key, value in data_dic1.items()}
    aux2 = {key: {tipo: value[tipo]} for key, value in data_dic2.items()}

    aux = aux1 | aux2
    
    all_aux = pd.DataFrame.from_dict( aux, orient='index').reset_index().rename(columns={'index': 'CEDULA_COD'})
    all_aux = all_aux.sort_values( by=["CEDULA_COD"], ascending=[True] )  
        
    data[colname] = data['CEDULA_COD'].map( all_aux.set_index('CEDULA_COD')[tipo] )

In [ ]:
data

In [ ]:
#Se agrega el número de cluster para el caso 1 con el método del codo
colname = 'NUM_CODO_1'
tipo = 'num_codo'
agregar_num_cluster( data, colname, ul_dic_cluster1, ml_dic_cluster1, tipo)

colname = 'NUM_SILUE_1'
tipo = 'max_silueta'
agregar_num_cluster( data, colname, ul_dic_cluster1, ml_dic_cluster1, tipo)

In [ ]:
data #442570 casos

In [ ]:
#Se agrega el número de cluster para el caso 2 con el método del codo
colname = 'NUM_CODO_2'
tipo = 'num_codo'
agregar_num_cluster( data, colname, ul_dic_cluster2, ml_dic_cluster2, tipo)

colname = 'NUM_SILUE_2'
tipo = 'max_silueta'
agregar_num_cluster( data, colname, ul_dic_cluster2, ml_dic_cluster2, tipo)

In [ ]:
data

In [ ]:
for clave in ul_dic_filt_si_nan:
    ul_dic_filt_si_nan[clave]['num_codo'] = -1
    ul_dic_filt_si_nan[clave]['max_silueta'] = -1
    
for clave in ml_dic_filt_si_nan:
    ml_dic_filt_si_nan[clave]['num_codo'] = -1
    ml_dic_filt_si_nan[clave]['max_silueta'] = -1
    
ul_dic_cluster3_all = ul_dic_cluster3 | ul_dic_filt_si_nan
ml_dic_cluster3_all = ml_dic_cluster3 | ml_dic_filt_si_nan

In [ ]:
#Se agrega el número de cluster para el caso 3 con el método del codo
colname = 'NUM_CODO_3'
tipo = 'num_codo'

agregar_num_cluster( data, colname, ul_dic_cluster3_all, ml_dic_cluster3_all, tipo)

colname = 'NUM_SILUE_3'
tipo = 'max_silueta'
agregar_num_cluster( data, colname, ul_dic_cluster3_all, ml_dic_cluster3_all, tipo)

In [ ]:
data

In [ ]:
ul_dic_cluster4_all = ul_dic_cluster4 | ul_dic_filt_si_nan
ml_dic_cluster4_all = ml_dic_cluster4 | ml_dic_filt_si_nan

In [ ]:
#Se agrega el número de cluster para el caso 4 con el método del codo
colname = 'NUM_CODO_4'
tipo = 'num_codo'
agregar_num_cluster( data, colname, ul_dic_cluster4_all, ml_dic_cluster4_all, tipo)

colname = 'NUM_SILUE_4'
tipo = 'max_silueta'
agregar_num_cluster( data, colname, ul_dic_cluster4_all, ml_dic_cluster4_all, tipo)

In [ ]:
data

In [ ]:
nombre_archivo = 'viu_k_mean_data_num_clust.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto9 = data

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto9, archivo) 

In [ ]:
#Se escojen dos cluster para trabajar
def clasificacion_kmean( data_dic, n, val, tipo, duplicados ):
    print('*' * 102)
    print('Clasificación de las observaciones según cluster')
    inicio = time.time()
    
    for ced in data_dic:
        aux = None
        
        if( duplicados =='si'):
            aux = np.array( data_dic[ced][ val ] ).reshape(-1, 1)
        if( duplicados =='no'):
            aux = np.unique( np.array( data_dic[ced][ val ] ) ).reshape(-1, 1)
            
        if aux.shape[0] >= n:
            
            kmeans = KMeans( n_clusters = n, init = 'k-means++', n_init = 1, random_state = semilla)
            clu = kmeans.fit_predict( aux ) + 1 
            ncl = np.unique( clu )
            centroide = np.array([[ np.nanmean( aux[clu == i], axis=0)[0], i ] for i in range(1, len(ncl) + 1)] ) 
            
            if len( ncl ) == 1:
                data_dic[ced][tipo] =  [-2] * len( aux ) #Valores unicos de aporte
            else:
                Q1 = np.quantile(aux, 0.25)
                Q3 = np.quantile(aux, 0.75)
                IQR = Q3-Q1
                LI = Q1 - 1.5 * IQR
                LS = Q3 + 1.5 * IQR

                cl_at = np.where( centroide[:,0] > (LS +  1e-8) )[0]

                mod_aux = np.zeros((len( aux ), 2) )
                mod_aux[:, 0] = aux[:, 0] 
                mod_aux[np.isin( clu, centroide[cl_at][:, 1]), 1] = 1  

                data_dic[ced][tipo] = mod_aux[:, 1].tolist()
            
        else:
            data_dic[ced][tipo] =  [-1] * len( aux )

    fin = time.time()  
    print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
    print('*' * 102)


In [ ]:
n = 2
val = 'SALARIO'
tipo = 'ATI_KM_M1'
duplicados = 'si'

clasificacion_kmean( ul_dic_cluster1, n, val, tipo, duplicados )
clasificacion_kmean( ml_dic_cluster1, n, val, tipo, duplicados )

In [ ]:
nombre_archivo = 'viu_k_mean_cluster1_ul_2.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto9 = ul_dic_cluster1

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto9, archivo) 
    
nombre_archivo = 'viu_k_mean_cluster1_ml_2.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto10 = ml_dic_cluster1

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto10, archivo) 
    

In [ ]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_k_mean_cluster1_ul_2.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    ul_dic_cluster1 = pickle.load( archivo ) 
    
nombre_archivo = 'viu_k_mean_cluster1_ml_2.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)
with open( ruta_archivo, 'rb') as archivo:
    ml_dic_cluster1 = pickle.load( archivo ) 

In [ ]:
#Depuración de casos atípicos
ul_dic_cluster1